In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from bitnotes import BitNotes
from pprint import pprint
import math
import constants as const

In [2]:
import pickle

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn
    
def load_noted_phrases(file_name: str) -> list:
    with open(file_name, 'rb') as f:
        noted_phrases = pickle.load(f)
        return noted_phrases

In [52]:
cxnn = load_cxnn('./data/cxnn_langs.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_pol.pickle')

In [ ]:
# cxnn = load_cxnn('./data/cxnn_monotone.pickle')
# noted_phrases = load_noted_phrases('./data/texts/tst_bbb.pickle')

In [53]:
%%time

cxnn.start_detection()

for num, phrase in enumerate(noted_phrases):
    bit_chord = cxnn.bit_notes.phrase_chord_as_bits(phrase)
    cxnn.detect_bits(bit_chord)
    if num % 500 == 0:
        print(f'phrase # {num}/{len(noted_phrases)}')

phrase # 0/2990
phrase # 500/2990
phrase # 1000/2990
phrase # 1500/2990
phrase # 2000/2990
phrase # 2500/2990
Wall time: 1min 17s


In [ ]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

In [54]:
detection = cxnn.summarize_detection()

0 621.9485002887619
1 187.90357759416048
2 111.0591452445269
3 231.88225369887314
4 398.80670096881886
5 133.678331074212
6 216.087074702922
7 248.31007118495648


In [50]:
cxnn.data_marks

{(0,): 'bel',
 (1,): 'blg',
 (2,): 'eng',
 (3,): 'epo',
 (4,): 'jbo',
 (5,): 'pol',
 (6,): 'rus',
 (7,): 'ukr'}

In [32]:
for bits, label in cxnn.data_marks.items():
    rate = np.sum(detection[list(bits)])
    print(label, rate)

bel 559.574733855
blg 178.632143638
eng 108.987736043
epo 183.595182381
jbo 353.674665459
pol 106.80813494
rus 204.339268233
ukr 274.450320353


In [57]:
cxnn.vectors_received

2990

In [56]:
print(cxnn.cluster_count())
points = [(bit, count) for bit, count in cxnn.point_stats() if count > 0]
# print(sorted(points, key=lambda x: x[0]))
bit_points = {}
for bit, count in points:
    bit_points[bit] = bit_points.get(bit, 0) + 1

print(sorted(bit_points.items()))
    
bit_clusters = {}
for bit, count in points:
    bit_clusters[bit] = bit_clusters.get(bit, 0) + count

print(sorted(bit_clusters.items()))

bit_activations = {}
for point in cxnn.point_objects:
    key = point.output_bit
    bit_activations[key] = bit_activations.get(key, 0) + sum(cluster.activation_count() for cluster in point.cluster_objects)

print(sorted(bit_activations.items()))

bit_divs = {}
for key in bit_clusters.keys():
    bit_divs[key] = bit_activations[key] / (bit_clusters[key] + bit_points[key])

pprint(sorted(bit_divs.items()))


19779
[(0, 56), (1, 34), (2, 33), (3, 51), (4, 55), (5, 27), (6, 32), (7, 40)]
[(0, 8310), (1, 1329), (2, 597), (3, 1767), (4, 4166), (5, 360), (6, 1007), (7, 2243)]
[(0, 1479975), (1, 247356), (2, 123496), (3, 460205), (4, 1049804), (5, 82021), (6, 243831), (7, 445890)]
[(0, 176.90353813052832),
 (1, 181.479090242113),
 (2, 196.02539682539683),
 (3, 253.13806380638064),
 (4, 248.70978441127696),
 (5, 211.9405684754522),
 (6, 234.67853705486044),
 (7, 195.30880420499344)]


In [11]:
sum([624, 342637, 0.0, 351069, 2021, 789, 350828, 2955, 274, 1810, 337907, 1010, 3099])

1395023.0

In [13]:
sum([556, 593, 1233, 3074, 4708, 689398, 24, 10971, 4258, 492475, 2502, 6425])

1216217

In [ ]:
[0.0, 0.0, 596, 268, 520, 81307, 0.0, 0.0, 0.0, 0.0, 310, 0.0, 79]
[0.0, 0.0, 678, 189, 268, 111389, 0.0, 0.0, 0.0, 0.0, 158, 0.0, 50]